In [2]:
from lusidtools.jupyter_tools import toggle_code

"""Time-variant Properties (e.g. coupon schedule) in LUSID 

Illustrates the use of time-variant properties, a type of property that depend on different effective dates.

Attributes
----------
coupon schedules
multi-valued properties
time-variant properties
"""

toggle_code("Toggle Docstring")

## Time-variant Properties

This notebook illustrates the use of time-variant properties, which are a type of property that depend on different effective dates. 

In the example below we use a quarterly ratings schedule as a demonstrative example, showing how the LUSID API can be used to query values on different effective dates. We will also demonstrate the [**bi-temporality**](https://support.finbourne.com/what-is-bi-temporal-data) of the data, using different [**'asAt'**](https://support.finbourne.com/what-is-asat-time) dates.  

In [2]:
# Import lusid specific packages
# These are the core lusid packages for interacting with the API via Python

import lusid
import lusid.models as models
from lusid.utilities import ApiClientFactory
from lusidjam.refreshing_token import RefreshingToken
from lusidtools.cocoon.cocoon import load_from_data_frame
from lusidtools.cocoon.cocoon_printer import (
    format_instruments_response,
    format_portfolios_response,
    format_transactions_response,
    format_quotes_response,
)
from lusidtools.pandas_utils.lusid_pandas import lusid_response_to_data_frame

# Import libraries
from datetime import datetime, timedelta
import time
import pytz
import json
import os
import pandas as pd

# Set pandas dataframe display formatting
pd.set_option('display.max_columns', None)
pd.options.display.float_format = '{:,.2f}'.format

# Configure notebook logging and warnings
import logging
logging.basicConfig(level=logging.INFO)

# Authenticate our user and create our API client
secrets_path = os.getenv("FBN_SECRETS_PATH")

# Initiate an API Factory which is the client side object for interacting with LUSID APIs
api_factory = lusid.utilities.ApiClientFactory(
    token=RefreshingToken(),
    api_secrets_filename = secrets_path,
    app_name="LusidJupyterNotebook")

# Import required LUSID APIs
property_definitions_api = api_factory.build(lusid.api.PropertyDefinitionsApi)
instruments_api = api_factory.build(lusid.api.InstrumentsApi)

print('LUSID Environment Initialised')
print('LUSID version : ', api_factory.build(lusid.api.ApplicationMetadataApi).get_lusid_versions().build_version)

LUSID Environment Initialised
LUSID version :  0.6.5656.0


## 1. Load Data

### 1.1 Instruments

Load the instruments data from the source file and upload them into LUSID. The dataset in the example containins large cap stocks from the FTSE 100 Index, which we can use as examples to later setup our properties to. 

In [3]:
# Read the instruments data
df = pd.read_csv("data/equity_transactions_isin.csv").drop_duplicates()
df.head()

,portfolio_code,portfolio_name,portfolio_base_currency,ISIN,sedol,instrument_type,instrument_id,name,txn_id,txn_type,txn_trade_date,txn_settle_date,txn_units,txn_price,txn_consideration,currency,strategy,cash_transactions
0,EQUITY_UK,LUSID's top 10 FTSE stock portfolio,GBP,GB0002162385,0216238,equity,EQ_1234,Aviva,txn-1,StockIn,02/01/2020,04/01/2020,120000,4.23,600000,GBP,ftse_tracker,nan
1,EQUITY_UK,LUSID's top 10 FTSE stock portfolio,GBP,GB00BH0P3Z91,BH0P3Z9,equity,EQ_1235,BHP,txn-2,StockIn,02/01/2020,04/01/2020,60000,17.89,1080000,GBP,ftse_tracker,nan
2,EQUITY_UK,LUSID's top 10 FTSE stock portfolio,GBP,GB0031348658,3134865,equity,EQ_1236,Barclays,txn-3,StockIn,02/01/2020,04/01/2020,150000,1.80,300000,GBP,ftse_tracker,nan
3,EQUITY_UK,LUSID's top 10 FTSE stock portfolio,GBP,GB0007980591,0798059,equity,EQ_1237,BP,txn-4,StockIn,02/01/2020,04/01/2020,100000,4.75,500000,GBP,ftse_tracker,nan
4,EQUITY_UK,LUSID's top 10 FTSE stock portfolio,GBP,GB0005405286,0540528,equity,EQ_1238,HSBC,txn-5,StockIn,02/01/2020,04/01/2020,20000,5.89,120000,GBP,ftse_tracker,nan


In [4]:
# Create a mapping schema for the instruments in the portfolio
instrument_mapping = {
    "identifier_mapping": {
        "ClientInternal": "ISIN"
    },
    "required": {
        "name": "name"
    },
}

# Instruments can be loaded using a dataframe with file_type set to "instruments"
result = load_from_data_frame(
    api_factory=api_factory,
    scope="TimeVariant",
    data_frame=df,
    mapping_required=instrument_mapping["required"],
    mapping_optional={},
    file_type="instruments",
    identifier_mapping=instrument_mapping["identifier_mapping"],
)

succ, failed, errors = format_instruments_response(result)
pd.DataFrame(data=[{"success": len(succ), "failed": len(failed), "errors": len(errors)}])

,success,failed,errors
0,10,0,0


### 1.2 Create the time-variant properties

With the instruments in LUSID, we can now define the properties and add them to the selected instruments. To this end we will need to define a [**property definition**](https://support.finbourne.com/what-is-a-property-definition), where we also need to specify the `life time` of the property as `TimeVariant`.  

In [5]:
# Setup the property details
property_scope = "TimeVariantProperty"
property_code = "QuarterlyRating"

def create_property(property_scope, property_code):
    # Create the property definition request
    property_definition = models.CreatePropertyDefinitionRequest(
                domain="Instrument",
                scope=property_scope,
                code=property_code,
                display_name="Quarterly Ratings Estimates",
                data_type_id=lusid.ResourceId(scope="system", code="number"),
                life_time="TimeVariant",
            )

    # create property definition
    try:
        property_definition_request = property_definitions_api.create_property_definition(
            create_property_definition_request=property_definition
        )

    except lusid.ApiException as e:
        if json.loads(e.body)["name"] == "PropertyAlreadyExists":
            logging.info(
                f"Property {property_definition.domain}/{property_definition.scope}/{property_definition.code} already exists"
            )
    return property_definition

# Pass our property scope and code to the property_definition
property_definition = create_property(property_scope, property_code)

INFO:root:Property Instrument/TimeVariantProperty/QuarterlyRating already exists


### 1.3 Upsert the instrument properties

In order to upsert the instrument property, we create a function defining the body of our property request and use the `InstrumentsApi` to pass the values for our instrument.For more details see [**upsert_instruments_properties**](https://www.lusid.com/docs/api/#operation/UpsertInstrumentsProperties).

In this case we've used Aviva as an example, to which we are passing dates and numeric values to using the example schedule found below. The dates will be stored in the property's `effective_from` parameter as seen below.

In [6]:
# set the property key using the property_definition -- this will follow the format domain/scope/code
property_key = f"{property_definition.domain}/{property_definition.scope}/{property_definition.code}"

# create a function to upsert the properties for a selected instrument using instruments_api
def upsert_instrument_property(ISIN, value, property_key, effectiveFrom):
    property_request = [
        models.UpsertInstrumentPropertyRequest(
            identifier_type="ClientInternal",
            identifier=ISIN,
            properties=[
                models.ModelProperty(
                    key=property_key,
                    value=models.PropertyValue(
                        metric_value=models.MetricValue(
                            value=value
                        )
                    ),
                    effective_from=effectiveFrom
                )
            ]
        )
    ]

    response = instruments_api.upsert_instruments_properties(
                        upsert_instrument_property_request=property_request)
    print(f'Upserted property for ISIN: {instrument_id}, with Value: {element[key]} and Eff. Date: {date}')
    return response

# pass in the schedule effective dates and values for a selected ISIN
instrument_id = "GB0002162385"
schedule = [
    { "2020-12-31" : "5"},
    { "2021-03-31" : "4"},
    { "2021-06-30" : "3"},
    { "2021-09-30" : "3"},
]


for element in schedule:
    for key in element:
        date = datetime.strptime(key, "%Y-%m-%d").astimezone(pytz.utc)
        upsert_instrument_property(instrument_id, float(element[key]), property_key, effectiveFrom=date)

Upserted property for ISIN: GB0002162385, with Value: 5 and Eff. Date: 2020-12-31 00:00:00+00:00
Upserted property for ISIN: GB0002162385, with Value: 4 and Eff. Date: 2021-03-30 23:00:00+00:00
Upserted property for ISIN: GB0002162385, with Value: 3 and Eff. Date: 2021-06-29 23:00:00+00:00
Upserted property for ISIN: GB0002162385, with Value: 3 and Eff. Date: 2021-09-29 23:00:00+00:00


## 2. Querying Instrument Properties

### 2.1 Get properties by effective dates

We can use the [**get_instruments**](https://www.lusid.com/docs/api#operation/GetInstruments) call to the API along with the instrument property key in order to query the properties attached to the instrument for a given effective date. 

In [7]:
# Create a function to query properties for a given instrument_id and effective date
def get_properties(ISIN, property_key, effective_date):
    response = instruments_api.get_instruments(
        identifier_type="ClientInternal",
        request_body=[ISIN],
        property_keys=[property_key],
        effective_at=datetime.strptime(effective_date, "%Y-%m-%d").astimezone(pytz.utc).isoformat(),
    )
    # return the effective date and property value
    return (response.values[ISIN].properties[0].effective_from,
            response.values[ISIN].properties[0].value.metric_value.value)

dates=["2020-12-31",
       "2021-03-31",
       "2021-06-30",
       "2021-09-30"]

data = [get_properties(instrument_id, property_key, date) for date in dates]
df = pd.DataFrame(data, columns=['Effective Date', 'Value'])
df['Effective Date'] = pd.to_datetime(df["Effective Date"].dt.strftime('%m/%d/%Y %I:%M:%S %p')) 
display(df)

,Effective Date,Value
0,2020-12-31 00:00:00,5.00
1,2021-03-30 23:00:00,4.00
2,2021-06-29 23:00:00,3.00
3,2021-06-29 23:00:00,3.00


### 2.2 Viewing properties by different  _'as at'_ dates

The below is an example of the bi-temporality of data stored in LUSID, which means that we can view data as it was on a certain 'as at' date. In this case, we will assume that there was an update to the entries with our previous ratings example, requiring an amendment to the property values. 

Given the bi-temporal feature, we will still be able to view the original entries as they were before this amendment was made. In this example, we update the expected ratings using a hypothetical downgrade.  

In [8]:
# Create a function to query properties for a given instrument_id and effective date
def get_properties_as_at(ISIN, effective_date, as_at_date):
    response = instruments_api.get_instruments(
        identifier_type="ClientInternal",
        request_body=[ISIN],
        property_keys=[property_key],
        effective_at=datetime.strptime(key, "%Y-%m-%d").astimezone(pytz.utc).isoformat(),
        as_at = as_at_date
    )
    # return the effective date and property value
    return (response.values[ISIN].properties[0].effective_from,
            as_at_date,
            response.values[ISIN].properties[0].value.metric_value.value)

In [9]:
# We begin by storing the current asAt value for '2021-06-30' before the change is made
time_now = datetime.now().astimezone(pytz.utc) 
data_1 = [get_properties_as_at(instrument_id, "2021-06-30", time_now)]
df_1 = pd.DataFrame(data_1, columns=['Effective Date', 'As at Date','Value'])
df_1['Effective Date'] = pd.to_datetime(df_1['Effective Date'].dt.strftime('%m/%d/%Y %I:%M:%S %p'))
df_1['As at Date'] = pd.to_datetime(df_1['As at Date'].dt.strftime('%m/%d/%Y %I:%M:%S %p'))


# Upsert the new set of data for the same instrument used before
new_schedule = [
    { "2020-12-31" : "5"},
    { "2021-03-31" : "2"},
    { "2021-06-30" : "1"},
    { "2021-09-30" : "1"},
]

for element in new_schedule:
    for key in element:
        date = datetime.strptime(key, "%Y-%m-%d").astimezone(pytz.utc)
        upsert_instrument_property(instrument_id, float(element[key]), property_key, effectiveFrom=date)

Upserted property for ISIN: GB0002162385, with Value: 5 and Eff. Date: 2020-12-31 00:00:00+00:00
Upserted property for ISIN: GB0002162385, with Value: 2 and Eff. Date: 2021-03-30 23:00:00+00:00
Upserted property for ISIN: GB0002162385, with Value: 1 and Eff. Date: 2021-06-29 23:00:00+00:00
Upserted property for ISIN: GB0002162385, with Value: 1 and Eff. Date: 2021-09-29 23:00:00+00:00


The [**get_instruments**](https://www.lusid.com/docs/api#operation/GetInstruments) call from the instruments API can be queried with the additional _'as at'_ parameter, which we will add to the get properties function below.  

In [10]:
# Set the new as_at time to illustrate the updated value for the same effective_date
time_now = datetime.now().astimezone(pytz.utc) 

# Create a dataframe with the new updated time
data_2 = [get_properties_as_at(instrument_id, "2021-06-30", time_now)]
df_2 = pd.DataFrame(data_2, columns=['Effective Date', 'As at Date','Value'])
df_2['Effective Date'] = pd.to_datetime(df_2['Effective Date'].dt.strftime('%m/%d/%Y %I:%M:%S %p'))
df_2['As at Date'] = pd.to_datetime(df_2['As at Date'].dt.strftime('%m/%d/%Y %I:%M:%S %p'))


In [11]:
# Show value for different as_at times
display(df_1)
display(df_2)

,Effective Date,As at Date,Value
0,2021-06-29 23:00:00,2020-10-28 21:00:03,3.00


,Effective Date,As at Date,Value
0,2021-06-29 23:00:00,2020-10-28 21:00:04,1.00
